In [22]:
import ipyparallel as ipp

In [23]:
# 需要使用 ipcluster start -n 4 启动四个引擎

c = ipp.Client()

TimeoutError: Hub connection request timed out

In [9]:
print(c.ids)

[0, 1, 2, 3]


In [13]:
c[:].apply_sync(lambda: "Hello High Performance Pythonistas!")

['Hello High Performance Pythonistas!',
 'Hello High Performance Pythonistas!',
 'Hello High Performance Pythonistas!',
 'Hello High Performance Pythonistas!']

In [14]:
dview=c[:] 

In [15]:
with dview.sync_imports():
    import os

importing os on engine(s)


In [17]:
dview.apply_sync(lambda:os.getpid())

[24584, 25224, 4816, 19468]

In [18]:
dview.execute("import sys")  # 远程执行命令的另一种方法

<AsyncResult: execute>

In [19]:
dview.push({'shared_data':[50, 100]})

<AsyncResult: _push>

In [20]:
dview.apply_sync(lambda:len(shared_data))

[2, 2, 2, 2]

In [21]:
import time
import ipyparallel as ipp
from ipyparallel import require


@require('random')
def estimate_nbr_points_in_quarter_circle(nbr_estimates):
    """Monte carlo estimate of the number of points in a      
       quarter circle using pure Python"""
    print(f"Executing estimate_nbr_points_in_quarter_circlewith {nbr_estimates:,} on pid {os.getpid()}")   
    nbr_trials_in_quarter_unit_circle = 0
    for step in range(int(nbr_estimates)):
        x = random.uniform(0, 1)
        y = random.uniform(0, 1)
        is_in_unit_circle = x * x + y * y <= 1.0
        nbr_trials_in_quarter_unit_circle += is_in_unit_circle
    return nbr_trials_in_quarter_unit_circle


if __name__ == "__main__":
    c = ipp.Client()
    nbr_engines = len(c.ids)
    print("We're using {} engines".format(nbr_engines))
    nbr_samples_in_total = 1e8
    nbr_parallel_blocks = 4

    dview = c[:]

    nbr_samples_per_worker = nbr_samples_in_total / nbr_parallel_blocks
    t1 = time.time()
    nbr_in_quarter_unit_circles = dview.apply_sync(estimate_nbr_points_in_quarter_circle, \
                                                   nbr_samples_per_worker)
    print("Estimates made:", nbr_in_quarter_unit_circles)

    nbr_jobs = len(nbr_in_quarter_unit_circles)
    pi_estimate = sum(nbr_in_quarter_unit_circles) * 4 / nbr_samples_in_total
    print("Estimated pi", pi_estimate)
    print("Delta:", time.time() - t1)

We're using 4 engines
Estimates made: [19633895, 19635459, 19630845, 19642520]
Estimated pi 3.14170876
Delta: 26.579152584075928


In [24]:
import dask.array as ds

In [25]:
x = ds.ones((10000,10000),chunks=(1000,1000)) #创建1Wx1W的数组,并分成1000X1000块
x

dask.array<ones, shape=(10000, 10000), dtype=float64, chunksize=(1000, 1000), chunktype=numpy.ndarray>

In [27]:
y = x + x.T
z = x[::2,:].mean(axis =1) #像 numpy一样操作数组

In [28]:
x.compute()#运行 compute()之后才能得到结果

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])